<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [13]:
km = {(1,2):3, (2,3):5, (9,8):17}

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame.from_dict(km, orient='index')

In [16]:
df

,0
"(1, 2)",3
"(2, 3)",5
"(9, 8)",17


In [17]:
df.index = pd.MultiIndex.from_tuples(km.keys())

In [18]:
df

,,0
1,2,3
2,3,5
9,8,17


In [19]:
pd.Series(km).rename_axis(['trip_id', 'refuel_at']).reset_index(name='fuel_km')

,trip_id,refuel_at,fuel_km
0,1,2,3
1,2,3,5
2,9,8,17


In [20]:
import csv
import os
from collections import defaultdict
# import gurobipy as grb
from gurobipy import *

# CONFIGURATION PARAMETERS
from config import VEH_FULL_RANGE, VEH_START_RANGE, SCRATCH_FOLDER, INPUT_FOLDER, OUTPUT_FOLDER
nodenode_dist = {}  # a dictionary describing distances between nodes {(from_node, to_node): dist_km}


# LOAD DATA & CONSTRUCT THE NETWORK
## Load distances between node pairs
with open(os.path.join(SCRATCH_FOLDER, 'NodeNode_Dists.csv'), newline='') as csvfile:
    distreader = csv.reader(csvfile)
    next(distreader, None)  # skip the header line
    for row in distreader:
        from_node_id, to_node_id, dist_km = row[0], row[1], int(row[2])
        nodenode_dist[(from_node_id, to_node_id)] = dist_km


## Load trips as sequences of node ids
trips_nodes = {}    # a dict of lists {trip_id: [(node_seq, node_id), (node_seq, node_id), ...]}. Describes which nodes are on which trips, and in what sequence
all_node_ids = set()
all_trip_ids = set()
with open(os.path.join(SCRATCH_FOLDER, 'Trips_nodes.csv'), newline='') as csvfile:
    tripreader = csv.reader(csvfile)
    next(tripreader, None)  # skip header line
    for row in tripreader:
        trip_id, node_id, node_seq = row[0], row[1], int(row[2])
        all_trip_ids.add(trip_id)
        all_node_ids.add(node_id)
        if trip_id in trips_nodes:
            trips_nodes[trip_id].append((node_seq, node_id))
        else:
            trips_nodes[trip_id] = [(node_seq, node_id)]
for trip in trips_nodes.values():   # sort every trip by node_seq
    trip.sort(key=lambda seq_node: seq_node[0])

print('Read input files... Found {} trips, involving {} nodes.'.format(len(all_trip_ids), len(all_node_ids)))


on_trip_covered_by_station = defaultdict(list)     
    # { (trip_id, dest_node_id): [refuel_node_id, refuel_node_id, ...] }
    # This dict describes on trip <trip_id>, which refueling locations can enable travel to node <dest_node_id>
    # Criteria for 'cover':
    # 1) dest_node is downstream of refuel_node, i.e. on a particular trip, the sequence index of dest_node is after refuel_node
    # 2) Distance from refuel_node to dest_node is no more than full_veh_range

for trip_id, trip_nodes in trips_nodes.items():
    n_nodes = len(trip_nodes)
    for refuel_index in range(n_nodes):
        for dest_index in range(refuel_index, n_nodes):
            if dest_index > refuel_index: # a station can only enable travels to its downstream
                refuel_node_id = trip_nodes[refuel_index][1]
                dest_node_id = trip_nodes[dest_index][1]
                if nodenode_dist[(refuel_node_id, dest_node_id)] >= VEH_FULL_RANGE:
                    break
                else:
                    on_trip_covered_by_station[(trip_id, dest_node_id)].append(refuel_node_id)




# OPTIMIZE
# Now we use Gurobi to build a BIP model and optimize it
## Create a Model
m = Model('LH_FCV_FRLM')  # FRLM = Flow Refueling Location Model

## Add variables
HRS_At = m.addVars(all_node_ids, vtype=GRB.BINARY, name='HRS_At')
    # Each node is associated with a 0-1 variable that denotes whether to place an HRS (1) or not (0)

## Add constraints
for trip_id, trip_nodes in trips_nodes.items():
    orig_node_id = trip_nodes[0][1]
    for node_index in range(1, len(trip_nodes)):
        node_id = trip_nodes[node_index][1]
        need_refuel = [f for f in on_trip_covered_by_station[trip_id, node_id] if (nodenode_dist[(orig_node_id, node_id)] >= VEH_START_RANGE)]
        # print('On trip {}, node {} need at least one station among nodes'.format(trip_id, node_id), need_refuel_at_least)
        if need_refuel:
            m.addConstr(quicksum(HRS_At[f] for f in need_refuel) >= 1)  # at least one refuel point needed to reach a node

## Set objective function
m.setObjective(quicksum(HRS_At[node_id] for node_id in all_node_ids), sense=GRB.MINIMIZE)

## Print model summary before optimizing
m.update()
grb_vars = m.getVars()
grb_cons = m.getConstrs()
print('Gurobi Model Summary:\nThere are {} variables in the model:'.format(len(grb_vars)))
print([v.VarName for v in grb_vars])
print('There are {} constraints:'.format(len(grb_cons)))
print([m.getRow(c) for c in grb_cons])

## Optimize it!
print('Optimizing...')
m.optimize()


## Show optimization results
print('We need stations at nodes:')
chosen_hrs_node_id = [node_id for node_id in all_node_ids if HRS_At[node_id].x>0 ]
print(chosen_hrs_node_id)

## Write optimization results to a CSV file
output_filename = 'Chosen_HRS_' + str(VEH_FULL_RANGE) + 'kmVehRange.csv'
# with open(output_filename, 'w', newline='') as csvfile:
#     hrswriter = csv.writer(csvfile, delimiter=',')
#     hrswriter.writerow(['hrs_node_id'])
#     for id in chosen_hrs_node_id:
#         hrswriter.writerow([id])


# Find which stations refuel which trips
on_trip_refuel_at = {}  # {trip_id: [node_id, node_id, ...]} on trip <trip_id>, refuel at nodes [node_id, node_id, ...]
trip_refuel_km = {}     # {(trip_id, refuel_at): refuel_km}  on trip <trip_id>, a vehicle refuels <refuel_km> range equivalent amount of fuel at node <refuel_at>
for trip_id, trip_nodes in trips_nodes.items():
    first_refuel = True
    print('trip_id:', trip_id)
    # print('\t{} nodes:'.format(len(trip_nodes)), trip_nodes)

    for node_seq, node_id in trip_nodes:    # visit every node on this trip
        if node_id in chosen_hrs_node_id:   # if this node has refueling, then refuel here
            if first_refuel:
                # print('\tRefuel at:', node_id, '(first refuel)')
                on_trip_refuel_at[trip_id] = [node_id]
                trip_refuel_km[(trip_id, node_id)] = VEH_FULL_RANGE - VEH_START_RANGE + nodenode_dist[trip_nodes[0][1], node_id]
                first_refuel = False
            else:
                prev_refuel_node = on_trip_refuel_at[trip_id][-1]
                # print('\tRefuel at:', node_id, 'prev refuel:', prev_refuel_node)
                on_trip_refuel_at[trip_id].append(node_id)
                trip_refuel_km[(trip_id, node_id)] = nodenode_dist[prev_refuel_node, node_id]

    if trip_id not in on_trip_refuel_at:    # if this trip does not need refuel (happens when it's shorter than VEH_START_RANGE)
        # print('\tTrip', trip_id, 'needs no refuel')
        on_trip_refuel_at[trip_id] = []

    print('\t{} refuels:'.format(len(on_trip_refuel_at[trip_id])),
          list(zip(on_trip_refuel_at[trip_id], [ trip_refuel_km[trip_id, refuel_at] for refuel_at in on_trip_refuel_at[trip_id]])))



import pandas as pd
km = pd.DataFrame.from_dict(trip_refuel_km, orient='index')

Read input files... Found 380 trips, involving 72 nodes.
Academic license - for non-commercial use only
Gurobi Model Summary:
There are 72 variables in the model:
['HRS_At[52]', 'HRS_At[44]', 'HRS_At[59]', 'HRS_At[64]', 'HRS_At[4]', 'HRS_At[23]', 'HRS_At[56]', 'HRS_At[26]', 'HRS_At[75]', 'HRS_At[41]', 'HRS_At[57]', 'HRS_At[27]', 'HRS_At[36]', 'HRS_At[39]', 'HRS_At[22]', 'HRS_At[31]', 'HRS_At[40]', 'HRS_At[42]', 'HRS_At[74]', 'HRS_At[2]', 'HRS_At[28]', 'HRS_At[14]', 'HRS_At[67]', 'HRS_At[60]', 'HRS_At[68]', 'HRS_At[65]', 'HRS_At[15]', 'HRS_At[66]', 'HRS_At[51]', 'HRS_At[19]', 'HRS_At[3]', 'HRS_At[73]', 'HRS_At[34]', 'HRS_At[1]', 'HRS_At[45]', 'HRS_At[53]', 'HRS_At[55]', 'HRS_At[54]', 'HRS_At[7]', 'HRS_At[10]', 'HRS_At[35]', 'HRS_At[13]', 'HRS_At[18]', 'HRS_At[58]', 'HRS_At[20]', 'HRS_At[37]', 'HRS_At[5]', 'HRS_At[46]', 'HRS_At[76]', 'HRS_At[32]', 'HRS_At[61]', 'HRS_At[63]', 'HRS_At[43]', 'HRS_At[48]', 'HRS_At[38]', 'HRS_At[70]', 'HRS_At[72]', 'HRS_At[11]', 'HRS_At[21]', 'HRS_At[25]', 'H

Variable types: 0 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 3326 rows and 62 columns
Presolve time: 0.01s
Presolved: 11 rows, 10 columns, 26 nonzeros
Found heuristic solution: objective 14.0000000
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: cutoff, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        14.00000   14.00000  0.00%     -    0s

Explored 0 nodes (8 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 14 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.400000000000e+01, best bound 1.400000000000e+01, gap 0.0000%
We 

trip_id: 318
	1 refuels: [('70', 184.0)]
trip_id: 319
	2 refuels: [('70', 184.0), ('34', 125)]
trip_id: 320
	2 refuels: [('70', 184.0), ('34', 125)]
trip_id: 321
	6 refuels: [('70', 184.0), ('34', 125), ('67', 122), ('44', 207), ('21', 111), ('6', 186)]
trip_id: 322
	6 refuels: [('70', 184.0), ('34', 125), ('67', 122), ('44', 207), ('21', 111), ('6', 186)]
trip_id: 323
	6 refuels: [('70', 184.0), ('34', 125), ('67', 122), ('44', 207), ('21', 111), ('6', 186)]
trip_id: 324
	2 refuels: [('4', 135.0), ('72', 131)]
trip_id: 325
	2 refuels: [('4', 135.0), ('11', 44)]
trip_id: 326
	2 refuels: [('4', 135.0), ('11', 44)]
trip_id: 327
	2 refuels: [('4', 135.0), ('11', 44)]
trip_id: 328
	2 refuels: [('4', 135.0), ('11', 44)]
trip_id: 329
	3 refuels: [('4', 135.0), ('11', 44), ('16', 120)]
trip_id: 330
	3 refuels: [('4', 135.0), ('11', 44), ('18', 107)]
trip_id: 331
	3 refuels: [('4', 135.0), ('11', 44), ('18', 107)]
trip_id: 332
	1 refuels: [('6', 137.0)]
trip_id: 333
	2 refuels: [('4', 135.0), 

In [29]:
trip_refuel = pd.Series(trip_refuel_km).rename_axis(['trip_id', 'refuel_at']).reset_index(name='fuel_km')
trip_refuel

,trip_id,refuel_at,fuel_km
0,1,72,216.0
1,1,11,162.0
2,2,72,216.0
3,2,11,162.0
4,3,72,216.0
5,3,11,162.0
6,4,72,216.0
7,4,11,162.0
8,5,72,216.0
9,5,11,162.0


In [32]:
trips_info = pd.read_csv(os.path.join(SCRATCH_FOLDER, 'Trips_info.csv'), usecols=['trip_id', 'ktons'], dtype={'trip_id': object})
trips_info

,trip_id,ktons
0,1,755.466557
1,2,755.466557
2,3,755.466557
3,4,755.466557
4,5,755.466557
5,6,755.466557
6,7,755.466557
7,8,755.466557
8,9,62227.594197
9,10,6886.451434


In [33]:
trip_refuel.merge(trips_info, left_on='trip_id', right_on='trip_id', how='left')

,trip_id,refuel_at,fuel_km,ktons
0,1,72,216.0,755.466557
1,1,11,162.0,755.466557
2,2,72,216.0,755.466557
3,2,11,162.0,755.466557
4,3,72,216.0,755.466557
5,3,11,162.0,755.466557
6,4,72,216.0,755.466557
7,4,11,162.0,755.466557
8,5,72,216.0,755.466557
9,5,11,162.0,755.466557
